# Python pour biochimistes: BioPython pour interroger des bases de données distantes

## Introduction

Dans un cahier précédent, nous avons développé la logique nécessaire pour accéder des ressources distantes: ouverture d'un flux de données (*handle*), construction d'un URL contenant les éléments de la requete, extraction de l'info, etc. C'est pas si complexe mais ça reste toujours un exercice long et pas toujours plaisant car ça demande de connaitre la façon de prendre contact avec la ressource distante. Pourquoi ne pas utiliser une approche orientée-objet pour faciliter notre vie: encapsuler les actions à prendre sous forme de méthodes avec un usage prévisible?

BioPython permet de faire ce type d'action et dans l'exercice qui suit, nous utiliserons `Bio.Entrez` pour intérroger directement Entrez. Noter que plusieurs autres sources de données sont aussi accessibles via BioPython; voir la documentation pour les détails. 

## Protocole pour obtenir des infos sur notre gène préféré

### Étape 1 - Quelles sont les bases de données accessibles via `Bio.Entrez` 

Accéder à une ressource distante demande un certain décorum... Par exemple, tel que déjà fait dans un exemple précédent, il faut généralement s'identifier. De plus, il faut souvent limiter ces demandes de requêtes pour ne pas inonder le serveur distant et risquer un bannissement. Ici, passer par BioPython est un avantage car Bio.Entrez force une limite de 3 requêtes/seconde.

In [1]:
#
# Primo, importons les classes nécessaires
#
import xml.dom.minidom
import xmltodict
from Bio import Entrez, SeqIO
#
# C'est toujours une bonne pratique de nous faire connaitre
#
Entrez.email = "sylvain.foisy@umontreal.ca"
#
# Création automatique du flux de données
#
handle = Entrez.einfo()
#
# Interception du résultat de la méthode einfo()
#
result = handle.read()
#
# On n'utilisera plus le format XML directement
# Si vous voulez voir la structure du XML,
# retirez les commentaires
#
# Impression du résultat et observation du format
#
#demoformat = xml.dom.minidom.parseString(result)
#print(demoformat.toprettyxml())
#
# Conversion du format XML en dictionnaire Python
#
aLib = xmltodict.parse(result)
#
# Quelles sont les bases de données accessibles?
#
print("Quelles sont les bases de donnees accessibles?")
for i in aLib["eInfoResult"]["DbList"]["DbName"]:
  #
  # On pourrait nettre i dans une liste pour réutilisation 
  # dans votre script
  #
  print(i)

Quelles sont les bases de donnees accessibles?
pubmed
protein
nuccore
ipg
nucleotide
structure
genome
annotinfo
assembly
bioproject
biosample
blastdbinfo
books
cdd
clinvar
gap
gapplus
grasp
dbvar
gene
gds
geoprofiles
medgen
mesh
nlmcatalog
omim
orgtrack
pmc
popset
proteinclusters
pcassay
protfam
pccompound
pcsubstance
seqannot
snp
sra
taxonomy
biocollections
gtr


### Étape 2 - Ok, on a une vue d'ensemble... Et ensuite? Recherche via Entrez.esearch

On voit que la liste des infos potentielles est vaste :-) Comment s'en tirer? Disons que nos voulons chercher des infos sur notre gène préféré: la cortactine humaine, alias CTTN.

In [2]:
#
# Cherchons dans NCBI Gene pour CTTN chez H sapiens, alias 9606 dans le jargon Taxid
#
aSearch = Entrez.esearch(db="gene", term="CTTN[Gene] AND 9606[Taxid]")
result = aSearch.read()
newLib = xmltodict.parse(result)
print(newLib)
#
# On voit que ça ne retourne qu'un seul résultat
# avec le geneId 2017
#
print("Quel est le geneID de CTTN chez H. sapiens?")
print(newLib["eSearchResult"]["IdList"]["Id"])

{'eSearchResult': {'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': {'Id': '2017'}, 'TranslationSet': None, 'TranslationStack': {'TermSet': [{'Term': 'CTTN[Gene]', 'Field': 'Gene', 'Count': '485', 'Explode': 'N'}, {'Term': '9606[Taxid]', 'Field': 'Taxid', 'Count': '359812', 'Explode': 'N'}], 'OP': 'AND'}, 'QueryTranslation': 'CTTN[Gene] AND 9606[Taxid]'}}
Quel est le geneID de CTTN chez H. sapiens?
2017


### Étape 3 - Récupération des infos via Entrez.efetch

Comme NCBI Gene est le dépositaire centralisé des infos sur les gènes recensés, récupérons dans un premier temps les info sur notre gène :-)

In [3]:
import json
#
# Utilisons Entrez.efetch
#
geneID = newLib["eSearchResult"]["IdList"]["Id"]
#
# On crée le handle pour géer le flux de données
#
summary = Entrez.esummary(db="gene", id=geneID)
#
# On crée un dictionnaire via la méthode read
# pour récupérer les données obtenues
#
summary = Entrez.read(summary)
#
# Juste pour présenter ça beau ;-)
# Essayer pour voir ;-)
# Ça permet de voir la structure du dictionnaire
#
#summaryJ = json.dumps(summary,indent=2)
#print(summaryJ)

print(summary["DocumentSummarySet"]["DocumentSummary"][0]["Name"])
print(summary["DocumentSummarySet"]["DocumentSummary"][0]["Description"])
print(summary["DocumentSummarySet"]["DocumentSummary"][0]["Summary"])

#
# Mais les données sur les transcrits et le protéines produites sont obtenues
# autrement....
#
data = Entrez.efetch(db="gene", id=geneID, rettype="gene_table", retmode="text")
data = data.read()
print(data)

CTTN
cortactin
This gene is overexpressed in breast cancer and squamous cell carcinomas of the head and neck. The encoded protein is localized in the cytoplasm and in areas of the cell-substratum contacts. This gene has two roles: (1) regulating the interactions between components of adherens-type junctions and (2) organizing the cytoskeleton and cell adhesion structures of epithelia and carcinoma cells. During apoptosis, the encoded protein is degraded in a caspase-dependent manner. The aberrant regulation of this gene contributes to tumor cell invasion and metastasis. Three splice variants that encode different isoforms have been identified for this gene. [provided by RefSeq, May 2010]
CTTN cortactin[Homo sapiens]
Gene ID: 2017, updated on 17-Jun-2024


Reference GRCh38.p14 Primary Assembly NC_000011.10  from: 70398529 to: 70436575
mRNA transcript variant 1 NM_005231.4, 18 exons,  total annotated spliced exon length: 3249
protein isoform a NP_005222.2 (CCDS41680.1), 16 coding  exons,

### Étape 4 - Obtenir les séquences pour chaque transcrit de la cortactine, 1ère partie

Il nous faut obtenir les identificateurs des transcrits afin d'aller récupérér les séquences. On récupère le contenu de la variable data sous la forme d'un tableau de lignes créé grâce à la méthode splitlines(). Cette dernière brise une longue chaine de caractères (le contenu de data) en ligne en recherchant les caractères de bris de ligne comme \n ou \r.

Après, il ne nous reste qu'à trier le tableau pour obtenir les infos nécessaires en récupérant l'information discriminante: NM_* ou XM_* (on pourrait faire de même pour les protéines)

In [4]:
allLines = data.splitlines()

allTranscripts = []

for aLine in allLines:
    if "mRNA transcript" in aLine:
        print(aLine)
        tmp = aLine.split(" ")
        #
        # Parce que l'on sait que le 5ème élément est
        # l'identificateur unique pour Genbank
        #
        # Il faut retirer la virgule à la fin...
        #
        allTranscripts.append(tmp[4].replace(',',''))

print(allTranscripts)

mRNA transcript variant 1 NM_005231.4, 18 exons,  total annotated spliced exon length: 3249
mRNA transcript variant 2 NM_138565.3, 17 exons,  total annotated spliced exon length: 3138
mRNA transcript variant 3 NM_001184740.2, 19 exons,  total annotated spliced exon length: 2255
mRNA transcript variant X1 XM_006718447.4, 16 exons,  total annotated spliced exon length: 3027
mRNA transcript variant X2 XM_006718448.5, 14 exons,  total annotated spliced exon length: 1601
mRNA transcript variant X3 XM_017017312.3, 11 exons,  total annotated spliced exon length: 1456
['NM_005231.4', 'NM_138565.3', 'NM_001184740.2', 'XM_006718447.4', 'XM_006718448.5', 'XM_017017312.3']


### Étape 5 - Obtenir les séquences pour chaque transcrit de la cortactine, 2ème partie

Ok, on a un tableau pour les identificateurs des séquences de chaque transcrit! Allons maintenant récupérer les infos avec `Entrez.efetch`.

In [5]:
#
# Nous allons procéder via une boucle for en itérant sur les items de la liste allTranscripts
# pour ne garder que ceux qui commencent par NM_
#
# Créons une lsite vide d'objets Entrez.Seq
#

mySequences = []

for i in allTranscripts:
    if "NM_" in i:
        fluxe = Entrez.efetch(db="nucleotide", id=i, rettype="gb", retmode="text")
        #
        # On se doit de transformer le flux de données en objet Entrez.Seq
        #
        aSeq = SeqIO.read(fluxe,"gb")
        mySequences.append(aSeq)
#
# Regardons uniquement les infos du 2ème transcrit
#
print(mySequences[1])
print(mySequences[1].seq)

ID: NM_138565.3
Name: NM_138565
Description: Homo sapiens cortactin (CTTN), transcript variant 2, mRNA
Number of features: 23
/molecule_type=mRNA
/topology=linear
/data_file_division=PRI
/date=07-APR-2024
/accessions=['NM_138565']
/sequence_version=3
/keywords=['RefSeq']
/source=Homo sapiens (human)
/organism=Homo sapiens
/taxonomy=['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
/references=[Reference(title='METTL3 promotes osteogenic differentiation of human umbilical cord mesenchymal stem cells by up-regulating m6A modification of circCTTN', ...), Reference(title='A MAP1B-cortactin-Tks5 axis regulates TNBC invasion and tumorigenesis', ...), Reference(title='Requirement of Site-Specific Tyrosine Phosphorylation of Cortactin in Retinal Neovascularization and Vascular Leakage', ...), Reference(title='Binding of USP4 to cortactin enhances cell migration

### Étape 6 - Écrire nos résultats dans des fichiers

Utilisons maintenant `SeqIO` pour écrire nos résultats dans des fichiers, un par objet `Seq`:

In [6]:
#
# SeqIO permet non seulement la lecture mais aussi l'écriture
# de fichiers dans divers formats
#
for i in mySequences:
    #
    # Profitons des méthodes de l'objet Seq pour déterminer le nom
    # du fichier
    #
    # Gardons notre fichier simple en utilisant le format FASTA
    #
    SeqIO.write(i,"../z.misc_files/data_seq/"+i.name+".fa","fasta")
